# Quantum QSVC Training, 
## Multiclass Performance with Single Label Training & Prediction, 

In [1]:
# For dataset.
from sklearn.datasets import make_multilabel_classification, make_classification
from qiskit_algorithms.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC, QSVC
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.kernels import BaseKernel, TrainableFidelityQuantumKernel

/var/folders/4z/946_ygn50p3bc_rsjbq9cg240000gn/T/ipykernel_23897/477652652.py:4: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import pandas as pd
import abc

In [7]:
# for data split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [6]:
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import RealAmplitudes

In [5]:
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel, FidelityStatevectorKernel

----

In [3]:
def conv_to_int(vector):
        val = "".join(vector.astype(str))
        value = int(val, 2)
        return value

In [4]:
def int_to_hot(x, no_of_classes):
        format = '{' + '0:0{:d}b'.format(no_of_classes) + '}'
        result = format.format(x)
        list_of_ints = [int(x) for x in result]
        return np.array(list_of_ints)

In [8]:
def get_mapping(no_of_classes):
        dictionary = {}
        numbers = list(range(0, 2**no_of_classes))
        for i in numbers:
            dictionary[i] = int_to_hot(i, no_of_classes)
        return dictionary

In [9]:
# get data
def data_generation(n_samples, n_features, n_classes, n_labels):
        """
        Generate classification data using sklearn's data generation.\
        """
        X, y = make_multilabel_classification(n_samples=n_samples, 
                                              n_features=n_features, 
                                              n_classes=n_classes, 
                                              n_labels=n_labels,
                                              random_state=algorithm_globals.random_seed
                                             )
        y_new = np.array([conv_to_int(val) for val in y])
        print(X.shape, y.shape, y_new.shape)
        print(X[:5], y[:5], y_new[:5])
        return X, y, y_new

In [13]:
results = {}
model = 'QSVC Kernel'
no_of_samples = 1024
no_of_features = [2,4,8,12] # as much as quantum side allows
no_of_classes = [3] 
no_of_labels = [3] 
no_of_featuremap_reps = [2,3,4,5,6,8,12]

In [14]:
for no_feature in no_of_features:
    for reps in no_of_featuremap_reps:
        for no_class in no_of_classes:
            for no_label in no_of_labels:
                if no_label <= no_class:
                    X, y, y_new = data_generation(no_of_samples, no_feature, no_class, no_label)
                    adhoc_feature_map = ZZFeatureMap(feature_dimension=no_feature, reps=reps, entanglement="linear")
                    adhoc_kernel = FidelityStatevectorKernel(feature_map=adhoc_feature_map)
                    optimizer = COBYLA(maxiter=50)
                    qsvc = QSVC(quantum_kernel=adhoc_kernel)
                    start = time.time()
                    qsvc.fit(X, y_new)
                    elapsed = time.time() - start
                    predictions = qsvc.predict(X)
                    mapping = get_mapping(no_class)
                    predictions_labels_final = np.array([mapping[val] for val in predictions])
                    print(predictions_labels_final[:5])
                    accuracy_scores = accuracy_score(y, predictions_labels_final)
                    f1_scores = f1_score(y, predictions_labels_final, average='weighted')
                    results.setdefault('Model', []).append(model)
                    results.setdefault('No of features', []).append(no_feature)
                    results.setdefault('No of samples', []).append(no_of_samples)
                    results.setdefault('No of classes', []).append(no_class)
                    results.setdefault('No of labels', []).append(no_label)
                    results.setdefault('No of feature map reps', []).append(reps)
                    results.setdefault('Accuracy', []).append(accuracy_scores)
                    results.setdefault('F1', []).append(f1_scores)
                    results.setdefault('Time taken', []).append(elapsed)

(1024, 2) (1024, 3) (1024,)
[[33. 20.]
 [32. 25.]
 [21. 25.]
 [28. 23.]
 [35. 24.]] [[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 1 1]] [7 7 7 7 3]
[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
(1024, 2) (1024, 3) (1024,)
[[31. 14.]
 [20. 14.]
 [37. 32.]
 [24. 27.]
 [25. 22.]] [[0 1 0]
 [1 1 0]
 [0 1 0]
 [1 1 0]
 [0 1 1]] [2 6 2 6 3]
[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
(1024, 2) (1024, 3) (1024,)
[[41. 10.]
 [25. 18.]
 [40. 14.]
 [52. 16.]
 [36. 19.]] [[0 1 0]
 [1 0 0]
 [1 1 1]
 [0 1 1]
 [1 1 1]] [2 4 7 3 7]
[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
(1024, 2) (1024, 3) (1024,)
[[31. 19.]
 [30. 18.]
 [32. 18.]
 [46.  0.]
 [42. 25.]] [[0 1 1]
 [1 1 1]
 [1 0 0]
 [0 0 1]
 [1 1 1]] [3 7 4 1 7]
[[1 1 1]
 [0 0 1]
 [1 1 1]
 [0 0 1]
 [0 0 1]]
(1024, 2) (1024, 3) (1024,)
[[16. 42.]
 [22. 36.]
 [10. 44.]
 [14. 34.]
 [10. 42.]] [[1 1 1]
 [1 1 0]
 [1 1 1]
 [0 1 1]
 [1 1 1]] [7 6 7 3 7]
[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
(1024, 2) (1024, 3) (1024,)
[[36. 16.]
 [18. 17.]
 [ 5. 45.]

In [15]:
df = pd.DataFrame(results)
df.to_csv(f'''results/Final/QSVC-experiments.csv''', index=False)